In [1]:
import pandas as pd
import numpy as np
import torch
import dgl
import dgl.function as fn
from dgl import DGLGraph
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import torch.nn as nn
import torch.optim as optim

In [2]:
df = pd.read_csv('soil_moisture.csv')

In [3]:
df = pd.read_csv('soil_moisture.csv')

sampled_df = df.sample(frac=0.10, random_state=42)

features = sampled_df[['Clay_0cm', 'Elevation', 'LST_Day_1km', 'LST_Night_1km', 'Precipitation', 'Sand_0cm',
                       'Soil_Moisture', 'SolarZenith', 'latitude', 'longitude', 'sur_refl_b01', 'sur_refl_b02',
                       'sur_refl_b03', 'sur_refl_b04', 'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07']].values

scaler = StandardScaler()
features = scaler.fit_transform(features)

n_nodes = 150
nodes = np.column_stack((np.random.uniform(sampled_df['latitude'].min(), sampled_df['latitude'].max(), n_nodes),
                         np.random.uniform(sampled_df['longitude'].min(), sampled_df['longitude'].max(), n_nodes)))

node_assignments = np.argmin(np.linalg.norm(nodes[:, None] - sampled_df[['latitude', 'longitude']].values, axis=2), axis=0)

In [4]:
g = DGLGraph()
g.add_nodes(n_nodes)
edges = []

for i in range(n_nodes):
    for j in range(i+1, n_nodes):
        dist = np.linalg.norm(nodes[i] - nodes[j])
        if dist < 500:
            edges.append((i, j))

src, dst = zip(*edges)
g.add_edges(src, dst)

/home/saagar/Coding/major_project/venv/lib/python3.10/site-packages/dgl/heterograph.py:92: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning(


In [5]:
train_idx, test_idx = train_test_split(np.arange(n_nodes), test_size=0.2, random_state=42)


In [6]:
targets = sampled_df['Soil_Moisture'].values

from sklearn.preprocessing import MinMaxScaler
scaler_target = MinMaxScaler()
targets = scaler_target.fit_transform(targets.reshape(-1, 1))


In [7]:
import torch

class GNNModel(nn.Module):
    def __init__(self, in_feats, hidden_size, out_feats):
        super(GNNModel, self).__init__()
        self.layers = nn.ModuleList([
            dgl.nn.GraphConv(in_feats, hidden_size),
            dgl.nn.GraphConv(hidden_size, out_feats)
        ])

    def forward(self, g, inputs):
        h = inputs
        for layer in self.layers:
            h = layer(g, h)
        return h

model = GNNModel(in_feats=features.shape[1], hidden_size=64, out_feats=1)

optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
import torch
import torch.optim as optim
import torch.nn as nn

criterion = nn.MSELoss()

lr = 0.001  # Learning rate

g = dgl.add_self_loop(g)

dgl.nn.GraphConv(features.shape[1], 1, allow_zero_in_degree=True)

def train(model, g, inputs, targets, train_idx, optimizer, criterion, epochs=100):
    for epoch in range(epochs):
        outputs = model(g, inputs)
        loss = criterion(outputs[train_idx], targets[train_idx])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}] Loss: {loss.item()}')

train(model, g, features, targets, train_idx, optimizer, criterion, 100)


TypeError: to() received an invalid combination of arguments - got (numpy.ndarray), but expected one of:
 * (torch.device device, torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (Tensor tensor, bool non_blocking, bool copy, *, torch.memory_format memory_format)
